In [2]:
from sturdy.pools import generate_assets_and_pools
from sturdy.protocol import AllocateAssets
from sturdy.utils.misc import greedy_allocation_algorithm
from sturdy.utils.lazy import lazy_allocation_algorithm

assets_and_pools = generate_assets_and_pools()
# print(f'pools: {assets_and_pools["pools"]}')
synapse = AllocateAssets(assets_and_pools=assets_and_pools)
greedy_allocations = greedy_allocation_algorithm(synapse=synapse)
# greedy_allocations = lazy_allocation_algorithm(synapse=synapse)
# Assert that all allocated amounts are more than equal to the minimum borrow amounts

all(
    amount >= assets_and_pools["pools"][pool_id]["borrow_amount"]
    for pool_id, amount in greedy_allocations.items()
)

True

In [24]:
import numpy as np

from sturdy.validator.reward import get_rewards


get_rewards(greedy_allocations, assets_and_pools)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3 from sturdy.validator.reward import get_rewards                                              │
│   4                                                                                              │
│   5                                                                                              │
│ ❱ 6 get_rewards(greedy_allocations, assets_and_pools)                                            │
│   7                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: get_rewards() missing 3 required positional arguments: 'uids', 'assets_and_pools', and 'responses'

In [3]:
%load_ext autoreload
%autoreload 2
from sturdy.utils.misc import calculate_apy
from sturdy.utils.lazy import lazy_and_humble_allocation_algorithm, lazy_allocation_algorithm
from sturdy.validator.reward import get_rewards
import sys

def get_apy(allocations, assets_and_pools):
    max_apy = sys.float_info.min
    alloc_yield = 0

    initial_balance = assets_and_pools["total_assets"]
    total_allocated = 0
    cheating = False

    for pool_id, allocation in allocations.items():
        pool = assets_and_pools["pools"][pool_id]
        total_allocated += allocation

        # reject allocations if miner is cheating somehow
        if total_allocated > initial_balance or allocation < pool["borrow_amount"]:
            # punish if miner they're cheating
            # TODO: create a more forgiving penalty system?
            raise Exception('Cheating')

        # calculate yield for given pool allocation
        util_rate = pool["borrow_amount"] / allocation
        interest_rate = calculate_apy(util_rate, pool)
        alloc_yield += allocation * interest_rate

    apy = alloc_yield / initial_balance

    if apy > max_apy:
        max_apy = apy

    return apy


current_allocations = lazy_allocation_algorithm(synapse=synapse)
humble_allocations = lazy_and_humble_allocation_algorithm(synapse=synapse)
print(f'Naive apy: {get_apy(current_allocations, assets_and_pools)}, ')
print(f'Greedy apy: {get_apy(greedy_allocations, assets_and_pools)}, ')
print(f'Humble apy: {get_apy(humble_allocations, assets_and_pools)}, ')


Naive apy: 0.19270409917000003, 
Greedy apy: 0.08575739930637451, 
Humble apy: 0.016344691899707817, 


In [11]:
current_allocations

{'0': 0.008,
 '1': 0.08600000000000001,
 '2': 0.036000000000000004,
 '3': 0.045,
 '4': 0.061,
 '5': 0.057,
 '6': 0.09,
 '7': 0.048,
 '8': 0.084,
 '9': 0.047}